In [5]:
# ssh gpu6 "source bulatov/memexp_env/bin/activate; cd bulatov/TXL/pytorch; bash run_wt103_base_memtrans_1by1.sh train --work_dir test_1by1 --device_ids 4 5 6 7 --data ~/bulatov/TXL/data/wt103"

In [3]:
x = torch.tensor([1.], requires_grad=True)
x.grad = torch.tensor([1.])

In [5]:
x.grad

tensor([1.])

In [7]:
l = x*2

In [33]:
torch.stack([torch.zeros(5, 4), torch.zeros(5, 4)]).sum(dim=0)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [20]:
l = torch.nn.functional.log_softmax(x, 2*x)

TypeError: log_softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (int dim, torch.dtype dtype)
 * (name dim, *, torch.dtype dtype)


In [17]:
print(l(x, x*2))
x.grad

tensor(1., grad_fn=<MseLossBackward0>)


tensor([1.])

In [18]:
l.backward()
x.grad

AttributeError: 'MSELoss' object has no attribute 'backward'

In [8]:
l.grad

/home/ayd98/anaconda3/envs/cudaenv/lib/python3.9/site-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [ ]:
# bash run_wt103_base.sh train --work_dir WORK_DIR --device_ids 0 1 --data PATH_TO_DATA --mem_len 0

In [1]:
# with open('/home/ayd98/Desktop/MIPT/TXL/data/enwik8/train.txt', 'r') as f:
#     res = f.read()

In [2]:
# import re
# re.sub(' +', ' ', 'The     quick brown    fox')
# res[:100]

In [8]:
# res = list(res)
# res = [s if s != ' ' else '\spce' for s in res]
# res = ' '.join(res)

In [3]:
# l = len(res)
# train = res[:int(0.9*l)]
# test = res[int(0.9*l):int(0.95*l)]
# val = res[int(0.95*l):]

In [4]:
# for var, name in zip([train, test, val], ['train', 'test', 'valid']):
#     with open(f'/home/ayd98/Desktop/MIPT/TXL/data/enwik8/{name}.txt', 'w') as f:
#         f.write(var)

In [5]:
# DATA_DIR=${DATA_ROOT}/pretrained_xl/tf_enwik8/data
# MODEL_DIR=${DATA_ROOT}/pretrained_xl/tf_enwik8/model

# # Model
# N_LAYER=24
# D_MODEL=1024
# D_EMBED=1024
# N_HEAD=8
# D_HEAD=128
# D_INNER=3072

# # Testing
# TEST_TGT_LEN=128
# TEST_MEM_LEN=3800
# TEST_CLAMP_LEN=1000

# TEST_CKPT_PATH=${MODEL_DIR}/model.ckpt-0
# TEST_BSZ=16
# TEST_NUM_CORE=2


# python data_utils.py \
#   --data_dir=$'data/enwik8' \
#   --dataset=enwik8 \
#   --tgt_len=${TEST_TGT_LEN} \
#   --per_host_test_bsz=${TEST_BSZ} \
#   --num_passes=1 \
#   --use_tpu=False

In [6]:
# import torchtext.datasets as datasets

# sets = datasets.WikiText103(root='data/wt103', split=('train', 'valid', 'test'))

# gen = sets[0]
# with open('data/wt103/train.txt', 'w') as f:
#     for i, t in enumerate(gen):
#         f.write(t)

# gen = sets[1]
# with open('data/wt103/valid.txt', 'w') as f:
#     for i, t in enumerate(gen):
#         f.write(t)

# gen = sets[2]
# with open('data/wt103/test.txt', 'w') as f:
#     for i, t in enumerate(gen):
#         f.write(t)

In [19]:
1 - int(True)

0

In [16]:
mask = torch.triu(torch.ones(6,8)*2, diagonal=3)
mask[2+1:, 2+1:] = 1
mask

tensor([[0., 0., 0., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 2., 2., 2.],
        [0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1.]])

In [4]:
(torch.ones(2) * 2 ).bool()

tensor([True, True])

In [2]:
import numpy as np
import time
import torch
import torch
import sys
sys.path.append('pytorch')
sys.path.append('pytorch/utils')

from pytorch.mem_transformer import *
from pytorch import data_utils

from experiment_utils.run_experiment import *
from experiment_utils.generate_data import *

## Variables

In [2]:
from sklearn.model_selection import ParameterGrid

TAG = '10tkn_len24_ext'

TASK_NAME = 'copy'
TRAIN_SIZE = 1000
VAL_SIZE = 200
TEST_SIZE = 100
NUM_INITS = 1


NUM_BATCHES = int(4e5)
BATCH_SIZE = 128
GENERATE_EVERY  = 10000
NUM_TOKENS = 10 + 2
ENC_SEQ_LEN = 24
DEC_SEQ_LEN = 48

INPUT_LEN = 24

#### Generate data

In [3]:
# # !mkdir data24
# np.random.seed(42)

# generator = copy_generator(batch_size=BATCH_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN, num_tokens=NUM_TOKENS)
# generate_data(generator, path=f'data{INPUT_LEN}', task_name=TASK_NAME, train_size=TRAIN_SIZE, test_size=TEST_SIZE, val_size=VAL_SIZE, batch_size=BATCH_SIZE)  

In [4]:
class data_loader:
    def __init__(self, task_name, path='data', batch_size=32, none_mask=True):
        self.X, self.y = np.load(f'{path}/{task_name}_X.npy'), np.load(f'{path}/{task_name}_y.npy')
        self.data_size = self.X.shape[0]
        self.data_ptr = 0

        if none_mask:
            self.src_mask, self.tgt_mask = None, None
        else:
            self.src_masks, self.tgt_mask = np.load(f'{path}/{task_name}_mask.npy'), None

        self.batch_size = batch_size
        self.none_mask = none_mask

    def __next__(self):
        X = self.X[self.data_ptr: self.data_ptr+self.batch_size]
        y = self.y[self.data_ptr: self.data_ptr+self.batch_size]
        
        if not self.none_mask:
            sm = self.src_masks[self.data_ptr: self.data_ptr+self.batch_size]
            sm = torch.tensor(sm).cuda()
        else:
            sm = None
            
        self.data_ptr = (self.data_ptr + self.batch_size) % self.data_size

        return torch.tensor(X),\
                torch.tensor(y),\
                sm, self.tgt_mask

### Run

In [5]:
model_parameters = ParameterGrid({
                'n_layer': [2],
                'n_head': [4],
                'd_head': [128],
                'num_mem_tokens': [0],
                'mem_len': [10]})

param = list(model_parameters)[0]

fixed_parameters = {'n_token': NUM_TOKENS,
                    'd_model': param['d_head'],# + param['num_mem_tokens']-1,
                    'd_inner': param['d_head'],
                    'dropout': 0,
                    'dropatt': 0,
                    'tie_weight': True,
                    'div_val': 1, # ???????
                    'tie_projs': [False],
                    'tgt_len': DEC_SEQ_LEN,
                    'ext_len': 0, 
                    'cutoffs': [],
                    'attn_type': 0,}

model = MemTransformerLM(**param, **fixed_parameters)#.cuda()

gen_train = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_train', batch_size=21)
src, tgt, _, _ = next(gen_train)
# src, tgt = src.cuda(), tgt.cuda()
src, tgt = src.cpu().T, tgt.cpu().T

mems = tuple()
# model(src, tgt.contiguous(), *mems)


In [6]:
with open('data/wt103/test.txt', 'r') as f:
    t = f.read()

In [7]:
t[:1000]

' \n = Robert Boulter = \n \n Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the <unk> Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n In 2006 , Boulter starred alongside Whishaw in the play Citizenship written by Mark Ravenhill . He appeared on

In [10]:
from data_utils import *

corpus = get_lm_corpus('data/wt103/', 'wt103')
ntokens = len(corpus.vocab)
args.n_token = ntokens

eval_batch_size = 10
tr_iter = corpus.get_iterator('train', args.batch_size, args.tgt_len,
    device=device, ext_len=args.ext_len)
va_iter = corpus.get_iterator('valid', eval_batch_size, args.eval_tgt_len,
    device=device, ext_len=args.ext_len)
te_iter = corpus.get_iterator('test', eval_batch_size, args.eval_tgt_len,
    device=device, ext_len=args.ext_len)

Producing dataset wt103...
building vocab with min_freq=0, max_size=None
final vocab size 267735 from 267734 unique tokens


NameError: name 'args' is not defined

In [19]:
it = corpus.get_iterator('train', 21, 150,
    device='cpu', ext_len=3)

In [20]:
lt = list(enumerate(it))

In [26]:
src, tgt, l = lt[0][1]
src[-10:, :5]

tensor([[     7,     24,   1072,   1483,   1944],
        [   286,      2,   3977,      2,      3],
        [     2,   1532,  38476,      1,    332],
        [  2377, 114037,      5,   7703,   1445],
        [    72,      0,  15128,  18109,     22],
        [     8,   1532,  37946,  12987,  46597],
        [   211,     24,   7426,  24601,   4597],
        [  1410,      2,     16,      5,   1233],
        [     4,   1532,      1,      1,      3],
        [     1,     24,    161,  20224,     21]])

In [28]:
src, tgt, l = lt[1][1]
src[:10, :5]

tensor([[  1410,      2,     16,      5,   1233],
        [     4,   1532,      1,      1,      3],
        [     1,     24,    161,  20224,     21],
        [   141,      2,     11,   1621,   4265],
        [   992,   1532,     38,   8866,     27],
        [    15,     24,    587,   8353,      8],
        [ 52174,      0,   1101,      1,    884],
        [ 11392,   1532,     17,    732,    118],
        [   358, 180578,     56,   3660,      4],
        [     3,      2,  16986,    334,   7138]])

In [38]:
src.shape

torch.Size([24, 21])

In [26]:
mems = model.init_mems(src.device)
res = model._forward(src, mems)

/home/ayd98/Desktop/MIPT/TXL/pytorch/mem_transformer.py:267: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1273.)
  attn_score = attn_score.float().masked_fill(


In [31]:
res[0].shape, [r.shape for r in res[1]]

(torch.Size([24, 21, 128]),
 [torch.Size([10, 21, 128]),
  torch.Size([10, 21, 128]),
  torch.Size([10, 21, 128])])

In [29]:
res[1]

[tensor([[[  6.8407,   8.3905,   0.9672,  ...,   8.8531,   5.1211, -14.8174],
          [  6.8407,   8.3905,   0.9672,  ...,   8.8531,   5.1211, -14.8174],
          [  6.8407,   8.3905,   0.9672,  ...,   8.8531,   5.1211, -14.8174],
          ...,
          [ -2.5426,  -9.4414,  -6.9474,  ...,  -8.7893,  -8.3210,  21.0291],
          [ -2.5426,  -9.4414,  -6.9474,  ...,  -8.7893,  -8.3210,  21.0291],
          [-24.5933,   0.5914,   0.3800,  ...,  -2.4394,  -3.9996, -29.2228]],
 
         [[  6.8407,   8.3905,   0.9672,  ...,   8.8531,   5.1211, -14.8174],
          [  0.8790,  -2.7545,   1.5977,  ...,  11.7079, -12.7973,  -4.8640],
          [  0.8790,  -2.7545,   1.5977,  ...,  11.7079, -12.7973,  -4.8640],
          ...,
          [ -1.6289, -18.3521,  -4.9785,  ...,   4.8849,  -1.5276,   0.5263],
          [ -4.0415,   0.3774,  11.2921,  ...,   6.9845,   7.1489,  -2.0019],
          [ -2.5426,  -9.4414,  -6.9474,  ...,  -8.7893,  -8.3210,  21.0291]],
 
         [[  0.8790,  -2.754

In [14]:
out.shape, mems.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [76]:
dp_model.module.num_mem_tokens

9

In [70]:
hasattr(model, "num_mem_tokens_")

False

In [19]:
src.shape

torch.Size([24, 21])

In [27]:
src.contiguous().view(-1).shape, src.reshape(-1).shape

(torch.Size([504]), torch.Size([504]))

In [15]:
model.layers[0].parameters()

<generator object Module.parameters at 0x7fc4ba0e0190>

In [6]:
torch.randn(1, 5)

tensor([[-0.9088, -0.4226, -0.1848, -0.7608, -0.4045]])

In [7]:
# self = model
# target = tgt.contiguous()
# data = src
# mem_tokens = None


# if not mems: mems = self.init_mems()
# if self.mem_tokens is None: self.init_mem_tokens()

# tgt_len = target.size(0)
# hidden, new_mems = self._forward(data, mems=mems, mem_tokens=mem_tokens)
# pred_hid = hidden[-tgt_len:]
# mem_tokens = hidden[-tgt_len - self.num_mem_tokens: -tgt_len]
# # if self.sample_softmax > 0 and self.training:
# #     assert self.tie_weight
# #     logit = sample_logits(self.word_emb,
# #         self.out_layer.bias, target, pred_hid, self.sampler)
# #     loss = -F.log_softmax(logit, -1)[:, :, 0]
# # else:
# loss = self.crit(pred_hid.view(-1, pred_hid.size(-1)), target.view(-1))
# loss = loss.view(tgt_len, -1)

# output = [loss]

# if new_mems is not None:
#     output += new_mems
# if self.num_mem_tokens not in (0, None):
#     output = [mem_tokens] + output

In [8]:
# tgt_len, hidden.shape, pred_hid.shape

In [9]:
# pred_hid.view(-1, pred_hid.size(-1))

In [10]:
# {**param, **fixed_parameters}

In [11]:
# out = model(src, src.contiguous())
# mem_tokens, loss, mems = out[0], out[1], out[2:]

In [12]:
class TransformerXL(MemTransformerLM):
    def __init__(self, enc_kwargs, dec_kwargs):
        super().__init__(**enc_kwargs)
        
        self.Encoder = MemTransformerLM(**enc_kwargs)
        self.Decoder = MemTransformerLM(**dec_kwargs)

        self.Encoder.init_mem_tokens()
        self.Decoder.init_mem_tokens()

        self.enc_kwargs = enc_kwargs
        self.dec_kwargs = dec_kwargs

    def forward(self, data, target, mems=None, mem_tokens=None):

        if not mems: mems = self.Encoder.init_mems()
        if self.Encoder.mem_tokens is None: self.Encoder.init_mem_tokens()

        tgt_len = target.size(0)
        hidden, enc_mems = self.Encoder._forward(src, mems=mems, mem_tokens=mem_tokens)

        pred_hid = hidden[-tgt_len:]
        mem_tokens = hidden[-tgt_len - self.Encoder.num_mem_tokens: -tgt_len]
        

        if not enc_mems: enc_mems = self.Encoder.init_mems()
        if self.Decoder.mem_tokens is None: self.Decoder.init_mem_tokens()
        
        tgt_st = tgt[:-1]
        tgt_gt = tgt[1: ]
        
        pred, dec_mems = self.Decoder._forward(tgt_st, mems=mems, mem_tokens=mem_tokens, context=pred_hid)

        return pred, dec_mems
        loss = Loss(pred, tgt_gt)
        # num_mem = self.enc_kwargs['num_mem_tokens']
        # if num_mem not in (None, 0):
        #     mem, enc = enc[:num_mem], enc[num_mem:]

        # if self.dec_kwargs['mem_len'] == 0:
        #     mems = None


        # start_tokens = tgt[:1, :]
        # out, new_mems = self.Decoder._forward(start_tokens)
        
        # # loss = self.crit(pred_hid.view(-1, pred_hid.size(-1)), target.view(-1))

        output = [loss]

        if new_mems is not None:
            output += new_mems
        if self.num_mem_tokens not in (0, None):
            output = [mem_tokens] + output
            
        return output


In [13]:
enc_param = {**param, **fixed_parameters}
dec_param = {**param, **fixed_parameters}

model = TransformerXL(enc_param, dec_param)

In [14]:
mems = tuple()
hidden, mems = model.Encoder._forward(src)

/home/ayd98/Desktop/MIPT/TXL/pytorch/mem_transformer.py:281: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1104.)
  attn_score = attn_score.float().masked_fill(


In [15]:
model(src, tgt)

RuntimeError: The size of tensor a (33) must match the size of tensor b (48) at non-singleton dimension 1

In [35]:
hidden.shape

torch.Size([33, 21, 128])

In [ ]:
gen_train = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE)
gen_val = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE)
gen_test = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE)


print_file = f'logs/{TASK_NAME}_{TAG}_memory_logs.txt'
t = time.time()
with torch.cuda.device(0):
    for init_num in range(NUM_INITS):
        with open(print_file, 'a') as f:
            f.write('\n\nInit number ' + str(init_num)+'\n')
        for i, param in enumerate(list(model_parameters)):
            with open(print_file, 'a') as f:
                f.write('\n\n' + str(param)+'\n')
            param['enc_depth'], param['enc_heads'] = param['depth,heads']
            param['dec_depth'], param['dec_heads'] = param['depth,heads']
            param.pop('depth,heads')

            with open(print_file, 'a') as f:
                f.write(f'{i / len(model_parameters) * 100}%')
            model = TransformerXL
            model = XTransformer(**param).cuda()

            model_name = f"{TASK_NAME}{INPUT_LEN}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_{TAG}_v{init_num}"

            optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
            train_validate_model(model, 
                            train_generator=gen_train, 
                            val_generator=gen_val, 
                            optim=optim, 
                            model_name=model_name, 
                            config=param,
                            num_batches=NUM_BATCHES,
                            generate_every=GENERATE_EVERY,
                            print_file=print_file,
                            tag=TAG,
                            overfit_stop=False)
            test_model(model, gen_test, model_name, param, TASK_NAME, tag=TAG)
            with open(print_file, 'a') as f:
                f.write(f'\nTotal time: {time.time() - t}\n')
            t = time.time()